In [1]:
import numpy as np
import torch 
import torch.nn as nn

In [2]:
# Parameters
N_paths = 10000   # number of simulated paths
N_steps = 30      # time steps
transaction_cost = 0.01  # 1% per trade
strike = 100
beta = 1          # mixed ES hyperparam


In [3]:
class HedgingLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # Output: position to hold

    def forward(self, x):
        # x: [batch_size, seq_len, input_dim]
        out, _ = self.lstm(x)
        position = self.fc(out)  # [batch_size, seq_len, 1]
        return position

In [ ]:
# 1. Generate sample paths for S_t
S_paths = simulate_paths(model="Heston", N_paths=N_paths, N_steps=N_steps, ...)

# 2. Neural network policy: delta_k = NN(S_{t_0}, ..., S_{t_k}) or NN(S_{t_k})
policy = HedgingLSTM()  # could be RNN or feedforward

# 3. Compute hedging P&L with transaction costs
def compute_pnl(S_path, NN, transaction_cost):
    n = len(S_path)
    deltas = []
    cash = 0
    delta_prev = 0
    for k in range(n):
        # Option 1: Use only S_{t_k}
        #delta_k = NN(S_path[k])
        # Option 2: Use RNN, so 
        delta_k = NN(S_path[:k+1])
        deltas.append(delta_k)
        # Trading cost
        trade = delta_k - delta_prev
        cost = transaction_cost * abs(trade) * S_path[k]
        cash -= trade * S_path[k] + cost
        delta_prev = delta_k
    # Terminal P&L: unwind all, pay option payoff
    payoff = max(S_path[-1] - strike, 0)
    pnl = cash + delta_prev * S_path[-1] - payoff
    return pnl

# 4. For all paths, compute losses (reverse P&L)
losses = []
for path in S_paths:
    pnl = compute_pnl(path, policy, transaction_cost)
    loss = -pnl  # reverse P&L = loss
    losses.append(loss)

# 5. Compute risk measure (mixed expected shortfall)
def mixed_es(losses, beta=1):
    L = np.sort(losses)
    n = len(L)
    es50 = np.mean(L[int(0.5*n):])
    es99 = np.mean(L[int(0.99*n):])
    return (es50 + beta * es99) / (1 + beta)

risk = mixed_es(losses, beta=1)

# 6. Train neural net to minimize risk
#     Use stochastic gradient descent (Adam) on NN parameters


SyntaxError: positional argument follows keyword argument (86231092.py, line 2)

In [ ]:
# Training loop:
for batch_S_paths in train_loader:
    deltas = model(batch_S_paths)
    # Compute P&L for each path, including proportional transaction costs
    # losses = [-pnl(path, deltas[i], transaction_cost) for i, path in enumerate(batch_S_paths)]
    # Compute mixed ES as loss
    loss = mixed_es(losses, beta)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
